## 测试验证收益分配算法 Part 2

In [63]:
from typing import List, Tuple
import pandas as pd


In [39]:
class ERC20(object):
    def __init__(self, name) -> None:
        self.address = name
        self._balance = {
            'alice': 0,
            'bob': 0,
            'chris': 0,
            'Trove': 0,
            'ConvexBooster': 0,
            'BaseRewardPool': 0,
            'staker': 0
        }
        self.totalSupply = 0
    
    def balanceOf(self, _who):
        return self._balance.get(_who, 0)

    def mint(self, _to, _amount):
        self.totalSupply += _amount
        self._balance[_to] += _amount
        
    def burn(self, _from, _amount):
        self.totalSupply -= _amount
        self._balance[_from] -= _amount
    
    def transfer(self, msgSender, _to, _amount):
        self._balance[msgSender] -= _amount
        self._balance[_to] += _amount
    
    def transferFrom(self, _from, _to, _amount):
        self._balance[_from] -= _amount
        self._balance[_to] += _amount
        

In [40]:
mim3Crv = ERC20('MIM-3LP3CRV-f')
cvxMim3Crv = ERC20('cvxMIM-3LP3CRV-f')

crv = ERC20('CRV')
cvx = ERC20('CVX')

In [48]:
class ConvexBooster(object):
    def __init__(self) -> None:
        self.address = 'ConvexBooster'
        self.staker = 'staker'
        self.poolInfo = {}
        
    def addBaseRewardPool(self, pid, token, lptoken, baseRewardPool):
        self.poolInfo = {
            pid: {
                'token': token,
                'lptoken': lptoken,
                'gauge': 'MIM-3LP3CRV-f-gauge',
                'crvRewards': baseRewardPool,
                'stash': 'ExtraRewardStashV3',
                'shutdown': False,
            }
        }
    
    def deposit(self, msgSender, _pid, _amount, _stake: bool):
        pool = self.poolInfo[_pid]
        # pool['lptoken'] transfer to staker(Voter)
        pool['lptoken'].transferFrom(msgSender, 'staker', _amount)
        # pool['gauge'] deposite lptoken
        
        
        if _stake:
            # mint for address(this)
            pool['token'].mint(self.address, _amount)
            # pool.crvRewards stakeFor
            pool['crvRewards'].stakeFor(self.address, msgSender, _amount)
            
    def withdrawTo(self, msgSender, _pid, _amount, _to):
        pool = self.poolInfo[_pid]
        pool['token'].burn(self.address, _amount)
        # pull from gauge if not shutdown
        stakerWithdraw = 0
        stashRewards = 0
        # lptoken transfer to `_to` 
        pool['lptoken'].transfer(self.address, _to, _amount)
        
    
class ConvexBaseRewardPool(object):
    def __init__(self, stakingToken, operator, rewardToken) -> None:
        self.address = 'BaseRewardPool'
        self.extraRewards = []
        self.totalSupply = 0
        self.balances = {
            'alice': 0,
            'bob': 0,
            'chris': 0,
            'Trove': 0,
            "BaseRewardPool": 0,
        }
        self.pid = 40
        self.rewardToken = rewardToken
        self.stakingToken = stakingToken
        self.operator = operator
        
        
    def stakeFor(self, msgSender, _for, _amount):
        for exR in self.extraRewards:
            exR.stake(_for, _amount)
        self.totalSupply += _amount
        self.balances[_for] += _amount
        # ConvexBooster's pid-lptoken transfer to this
        self.stakingToken.transferFrom(msgSender, self.address, _amount)
    
    def withdrawAndUnwrap(self, msgSender, _amount, _claim: bool):
        for exR in self.extraRewards:
            exR.withdraw(msgSender, _amount)
        
        self.totalSupply -= _amount
        self.balances[msgSender] -= _amount
        self.operator.withdrawTo(self.address, self.pid, _amount, msgSender)
        
        if _claim:
            self.getReward(msgSender, True)
        
    def getReward(self, msgSender, _claimExtras: bool):
        # reward = earned(msgSender)
        reward = 100
        if reward:
            reward[msgSender] = 0
            self.rewardToken.transfer(msgSender, reward)
        if _claimExtras:
            for exR in self.extraRewards:
                exR.getReward(msgSender)
        

In [49]:
booster = ConvexBooster()

baseRewardPool = ConvexBaseRewardPool(cvxMim3Crv, booster, crv)

booster.addBaseRewardPool(40, cvxMim3Crv, mim3Crv, baseRewardPool)


In [71]:

class Trove(object):
    
    def __init__(self, booster, baseRewardPool):
        self.address = 'Trove'
        self.userDebt = {
            "0x00": {
                "coll": 0,
                "borrow": 0,
            }
        }
        self.booster = booster
        self.baseRewardPool = baseRewardPool
        self.cvxAmount = 0
        self.crvAmount = 0
    
    def leverage(self, address, coll, borrowed):
        self.userDebt[address] = {'coll': coll, 'borrow': borrowed}
        print(f'before stake has {self.baseRewardPool.balances.get(self.address, 0)}')
        booster.deposit(self.address, 40, coll, True)
        print(f'before stake has {self.baseRewardPool.balances.get(self.address, 0)}')
        
    
    def deleverage(self, address):
        if self.userDebt[address]['coll'] > 0:
            (_1, _2, _3) = self.getReward(address)
            self.userDebt[address] = {'coll': 0, 'borrow': 0}
            print(f'{address} deleveraged, clear debt')
            print(f'get rewards: {_1, _2, _3}')
        else:
            print('[Warning] not deleverage')
    
    def getReward(self, address) -> List[int]:
        hasColl = self.userDebt[address]['coll']
        unstake_lptoken = hasColl * 0.99
        oldCvxAmount = self.cvxAmount
        oldCrvAmount = self.crvAmount
        cvx_balanceOf_this = 0
        crv_balanceOf_this = 0
        # distribute reward to user 
        crvRe = 0
        cvxRe = 0
        others = 0
        return (crvRe, cvxRe, others)

In [72]:
# alice bob chris

mim3Crv.mint('alice', 1000)
mim3Crv.mint('bob', 2000)
mim3Crv.mint('chris', 3000)

In [73]:


mim3crvTrove = Trove(booster, baseRewardPool)


In [80]:
mim3crvTrove.leverage('alice', 100, 200)

before stake has 1400
before stake has 1500


In [81]:
cvxMim3Crv.balanceOf('alice'), cvxMim3Crv.balanceOf('Trove')

(0, 0)

In [75]:
mim3crvTrove.leverage('bob', 200, 300)

before stake has 1000
before stake has 1200


In [76]:
mim3crvTrove.deleverage('alice')

alice deleveraged, clear debt
get rewards: (0, 0, 0)


In [77]:
mim3crvTrove.deleverage('alice')

[Warning] not deleverage


In [82]:
cvxMim3Crv.balanceOf('alice'), cvxMim3Crv.balanceOf('Trove')


(0, 0)